In [421]:
import numpy as np
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
%matplotlib inline

#### Step 0: select Mississippi river watersheds for this study

In [141]:
# Define geographical region to search for USGS stations
# These are the key words to put in the USGS search engine
watersheds = ['Missouri',
              'Yellowstone',
              'Platte',
              'Mississippi',
              'Illinois',
              'Ohio',
              'Cumberland',
              'Tennessee',
              'Arkansas',
              'Red']

#### Step 1: generate a station list for each geographical region (watershed)

In [23]:
# stations = dict.fromkeys(watersheds, {})

# for ws in watersheds:
#     url = ('https://waterdata.usgs.gov/nwis/inventory?search_station_nm='
#            +ws.split()[0]
#            +'%20river%20at&search_station_nm_match_type=beginning&data_type=rt&group_key=NONE&format=sitefile_output&sitefile_output_format=html_table&column_name=site_no&column_name=station_nm&column_name=dec_lat_va&column_name=dec_long_va&list_of_search_criteria=search_station_nm%2Cdata_type')
#     tables = pd.read_html(url) # pd.read_html() returns a list of tables from the url
#     df = tables[1]
#     df.columns = ['site_no','site_name','site_lat','site_lon','Cooraccr','latlongdatum']
#     stations[ws] = df.drop('Cooraccr',axis=1).set_index('site_no').to_dict(orient='index')
#     print('...%d stations were added to %s' %(len(df),ws))



...40 stations were added to Missouri
...8 stations were added to Yellowstone
...6 stations were added to Platte
...35 stations were added to Mississippi
...14 stations were added to Illinois
...40 stations were added to Ohio
...18 stations were added to Cumberland
...6 stations were added to Tennessee
...31 stations were added to Arkansas
...26 stations were added to Red


In [62]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import os
chromedriver = "/Users/sealoving/Documents/GitHub/Liang_Metis/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)

In [74]:
stations = {}

for ws in watersheds:
    url = ('https://waterdata.usgs.gov/nwis/inventory?search_station_nm='
           +ws.split()[0]
           +'%20river%20at&search_station_nm_match_type=beginning&data_type=rt&group_key=NONE&format=sitefile_output&sitefile_output_format=html_table&column_name=site_no&column_name=station_nm&column_name=dec_lat_va&column_name=dec_long_va&list_of_search_criteria=search_station_nm%2Cdata_type')
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    station_table = soup.find_all('table')[1].find_all('tr')[2:]
    
    for i in range(len(station_table)):
        site_no = station_table[i].find_all('td')[0].text.strip()
        name = station_table[i].find_all('td')[1].text.strip()
        lat = float(station_table[i].find_all('td')[2].text.strip())
        lon = float(station_table[i].find_all('td')[3].text.strip())
        stations[site_no] = {'watershed': ws,
                             'site_name': name,
                             'lat': lat,
                             'lon': lon}
    
    print('...%d stations were added to %s' %(len(station_table),ws))

...40 stations were added to Missouri
...8 stations were added to Yellowstone
...6 stations were added to Platte
...35 stations were added to Mississippi
...14 stations were added to Illinois
...40 stations were added to Ohio
...18 stations were added to Cumberland
...6 stations were added to Tennessee
...31 stations were added to Arkansas
...26 stations were added to Red


In [82]:
df = pd.DataFrame.from_dict(data=stations,orient='index')
df.head()

,watershed,site_name,lat,lon
02111500,Red,"REDDIES RIVER AT NORTH WILKESBORO, NC",36.175000,-81.168889
03085730,Ohio,Ohio River at Emsworth Dam Upper Pool @ Emsworth,40.503889,-80.085556
03085734,Ohio,Ohio River at Emsworth Dam Lower Pool @ Emsworth,40.505250,-80.089833
03086000,Ohio,"Ohio River at Sewickley, PA",40.549234,-80.205615
03086001,Ohio,"Ohio River (lower pool) at Sewickley, PA",40.549722,-80.206944


#### Step 2: populate data inventory for each station in the lists

In [139]:
# A function that retrieves data inventory information based on station "site_no" 
# "site_no" is the key in the sub-dictionary for each watershed in the "stations"
def get_data_info(soup,data_type):
    '''Grab a value from USGS data invetory table HTML
    
    Takes a string data type of a station on the page and
    returns the string in the next/next/next object (not siblings, unfortunately)
    or None if nothing is found.
    '''
    obj = soup.find(text=re.compile(data_type))
    if not obj: 
        return None
    D_start = obj.findNext().text.strip()
    D_end = obj.findNext().findNext().text.strip()
    D_count = obj.findNext().findNext().findNext().text.strip()
    return([D_start,D_end,D_count])


    
def get_inventory(stations,data_types):
    '''Populate data info for each station in stations dictionary
    
    data_types: a list of data type to be extracted
    such as: 'Discharge','Gage','Salinity','Turbidity'
    '''
    for site_no in stations.keys():
        url = 'https://waterdata.usgs.gov/nwis/inventory/?site_no='+site_no+'&agency_cd=USGS'

        response = requests.get(url)
#         print(response.status_code)
        soup = BeautifulSoup(response.text,"lxml")

        description_table = soup.find(id='stationTable').find_all('dl')[0]
        drainage = description_table.find(text=re.compile('Drainage'))
        if drainage:
            stations[site_no]['drainage_area_sqmi'] = float(drainage.split()[2].replace(',',''))
        else:
            stations[site_no]['drainage_area_sqmi'] = None
        datum = description_table.find(text=re.compile('Datum'))
        if datum:
            stations[site_no]['gage_datum_ft'] = float(datum.split()[3].replace(',',''))
        else:
            stations[site_no]['gage_datum_ft'] = None


        inventory_table = soup.find(id='stationTable').find_all('dl')[1]
        for data_type in data_types:
            data_info = get_data_info(inventory_table, data_type)
            stations[site_no][data_type] = data_info
            
    return(stations)

In [140]:
# Define the data type to be extracted
# only include discharge, gage height, and turbidity
data_types = ['Discharge','Gage','Turbidity']
stations = get_inventory(stations,data_types)

In [183]:
df = pd.DataFrame.from_dict(data=stations,orient='index')
# drop stations that have neither discharge nor gage data
df = df.dropna(axis=0,subset=['Discharge','Gage'],how='all')

df.head()

,watershed,site_name,lat,lon,drainage_area_sqmi,gage_datum_ft,Discharge,Gage,Turbidity
02111500,Red,"REDDIES RIVER AT NORTH WILKESBORO, NC",36.175000,-81.168889,89.2,978.62,"[2004-10-01, 2018-01-25, 38125]","[2004-10-01, 2018-01-25, 15598]",None
03086000,Ohio,"Ohio River at Sewickley, PA",40.549234,-80.205615,19500.0,680.00,"[1933-10-01, 2018-01-25, 30793]",None,None
03110685,Ohio,"OHIO R AT NEW CUMBERLAND LOCK & DAM (UPPER), OH",40.528399,-80.626740,23820.0,652.06,None,"[2010-09-30, 2018-01-25, 2659]",None
03110690,Ohio,"OHIO R AT NEW CUMBERLAND LOCK & DAM (LOWER), OH",40.528121,-80.625629,23820.0,631.56,None,"[2010-09-30, 2018-01-25, 2630]",None
03111515,Ohio,"OHIO R AT PIKE ISLAND DAM NR WHEELING (UPPER), WV",40.152849,-80.699802,24600.0,631.52,None,"[2010-09-30, 2018-01-25, 2662]",None


In [184]:
def expand_list(df, col):
    to_expand = df.dropna(axis=0, subset=[col])[col]
    expanded = pd.DataFrame(to_expand.tolist(),index=to_expand.index,columns=[col+'_start',col+'_end',col+'_count'])
    expanded[col+'_count'] = expanded[col+'_count'].astype(int)
    dfnew = pd.concat([df,expanded], axis=1)
    return dfnew
    
df = expand_list(df,'Discharge')
df = expand_list(df,'Gage')
df.head()

,watershed,site_name,lat,lon,drainage_area_sqmi,gage_datum_ft,Discharge,Gage,Turbidity,Discharge_start,Discharge_end,Discharge_count,Gage_start,Gage_end,Gage_count
02111500,Red,"REDDIES RIVER AT NORTH WILKESBORO, NC",36.175000,-81.168889,89.2,978.62,"[2004-10-01, 2018-01-25, 38125]","[2004-10-01, 2018-01-25, 15598]",None,2004-10-01,2018-01-25,38125.0,2004-10-01,2018-01-25,15598.0
03086000,Ohio,"Ohio River at Sewickley, PA",40.549234,-80.205615,19500.0,680.00,"[1933-10-01, 2018-01-25, 30793]",None,None,1933-10-01,2018-01-25,30793.0,NaN,NaN,NaN
03110685,Ohio,"OHIO R AT NEW CUMBERLAND LOCK & DAM (UPPER), OH",40.528399,-80.626740,23820.0,652.06,None,"[2010-09-30, 2018-01-25, 2659]",None,NaN,NaN,NaN,2010-09-30,2018-01-25,2659.0
03110690,Ohio,"OHIO R AT NEW CUMBERLAND LOCK & DAM (LOWER), OH",40.528121,-80.625629,23820.0,631.56,None,"[2010-09-30, 2018-01-25, 2630]",None,NaN,NaN,NaN,2010-09-30,2018-01-25,2630.0
03111515,Ohio,"OHIO R AT PIKE ISLAND DAM NR WHEELING (UPPER), WV",40.152849,-80.699802,24600.0,631.52,None,"[2010-09-30, 2018-01-25, 2662]",None,NaN,NaN,NaN,2010-09-30,2018-01-25,2662.0


In [185]:
df.loc['02111500','Discharge_count']

38125.0

In [186]:
df = df.drop(['Discharge','Gage'],axis=1)
df.head()

,watershed,site_name,lat,lon,drainage_area_sqmi,gage_datum_ft,Turbidity,Discharge_start,Discharge_end,Discharge_count,Gage_start,Gage_end,Gage_count
02111500,Red,"REDDIES RIVER AT NORTH WILKESBORO, NC",36.175000,-81.168889,89.2,978.62,None,2004-10-01,2018-01-25,38125.0,2004-10-01,2018-01-25,15598.0
03086000,Ohio,"Ohio River at Sewickley, PA",40.549234,-80.205615,19500.0,680.00,None,1933-10-01,2018-01-25,30793.0,NaN,NaN,NaN
03110685,Ohio,"OHIO R AT NEW CUMBERLAND LOCK & DAM (UPPER), OH",40.528399,-80.626740,23820.0,652.06,None,NaN,NaN,NaN,2010-09-30,2018-01-25,2659.0
03110690,Ohio,"OHIO R AT NEW CUMBERLAND LOCK & DAM (LOWER), OH",40.528121,-80.625629,23820.0,631.56,None,NaN,NaN,NaN,2010-09-30,2018-01-25,2630.0
03111515,Ohio,"OHIO R AT PIKE ISLAND DAM NR WHEELING (UPPER), WV",40.152849,-80.699802,24600.0,631.52,None,NaN,NaN,NaN,2010-09-30,2018-01-25,2662.0


In [187]:
df.to_pickle('USGS_stations.pickle')

#### Step 3: download station discharge/gage data for selected date range

In [188]:
df.dropna(axis=0,subset=['Turbidity'])[df.watershed=='Mississippi']

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,watershed,site_name,lat,lon,drainage_area_sqmi,gage_datum_ft,Turbidity,Discharge_start,Discharge_end,Discharge_count,Gage_start,Gage_end,Gage_count
05420500,Mississippi,"Mississippi River at Clinton, IA",41.780586,-90.252073,85600.0,562.68,"[2015-03-17, 2018-01-26, 951]",1873-06-02,2018-01-25,52765.0,NaN,NaN,NaN
07020850,Mississippi,"Mississippi River at Cape Girardeau, MO",37.301889,-89.518000,NaN,304.27,"[2015-01-28, 2018-01-25, 2727]",NaN,NaN,NaN,1984-10-01,2018-01-26,11781.0
07374000,Mississippi,"Mississippi River at Baton Rouge, LA",30.445667,-91.191556,1125810.0,0.00,"[2011-09-30, 2018-01-25, 4844]",2004-03-17,2018-01-25,4679.0,1997-07-02,2018-01-25,14456.0
07374525,Mississippi,"Mississippi River at Belle Chasse, LA",29.857151,-89.977847,1130000.0,-6.88,"[2012-09-30, 2018-01-25, 5310]",2008-10-29,2018-01-25,3324.0,2008-10-29,2018-01-25,3324.0
07374540,Mississippi,"Mississippi River at Buras, LA",29.357139,-89.523861,NaN,-0.84,"[2012-09-19, 2016-02-24, 3588]",NaN,NaN,NaN,2012-09-19,2016-02-24,3618.0


In [367]:
# Define the study time period
# Note that a "water year" starts on Oct.1st
# Dates are set to the maximum availability of downstream turbidity at Baton Rouge, LA
start_date = '2011-10-01' #six water-years
end_date = '2017-09-30'

In [368]:
df = pd.read_pickle('USGS_stations.pickle')
# find stations that have full range data and download them
df['url'] = None
df['type'] = None

for site_no in df.index:
    if ~np.isnan(df.loc[site_no].Discharge_count):
        if (df.loc[site_no].Discharge_start < start_date) and (df.loc[site_no].Discharge_end > end_date):
            url = ('https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no='+
                   site_no+'&referred_module=sw&period=&begin_date='+
                   start_date+'&end_date='+end_date)
            df.set_value(site_no, 'url', url)
            df.set_value(site_no, 'type', 'discharge')
    elif ~np.isnan(df.loc[site_no].Gage_count):
        if (df.loc[site_no].Gage_start < start_date) and (df.loc[site_no].Gage_end > end_date):
            url = ('https://waterdata.usgs.gov/nwis/dv?cb_00065=on&format=rdb&site_no='+
                   site_no+'&referred_module=sw&period=&begin_date='+
                   start_date+'&end_date='+end_date)
            df.set_value(site_no, 'url', url)
            df.set_value(site_no, 'type', 'gage')

df = df.dropna(subset=['url'],axis=0)
df.head()

,watershed,site_name,lat,lon,drainage_area_sqmi,gage_datum_ft,Turbidity,Discharge_start,Discharge_end,Discharge_count,Gage_start,Gage_end,Gage_count,url,type
02111500,Red,"REDDIES RIVER AT NORTH WILKESBORO, NC",36.175000,-81.168889,89.2,978.62,None,2004-10-01,2018-01-25,38125.0,2004-10-01,2018-01-25,15598.0,https://waterdata.usgs.gov/nwis/dv?cb_00060=on...,discharge
03086000,Ohio,"Ohio River at Sewickley, PA",40.549234,-80.205615,19500.0,680.00,None,1933-10-01,2018-01-25,30793.0,NaN,NaN,NaN,https://waterdata.usgs.gov/nwis/dv?cb_00060=on...,discharge
03110685,Ohio,"OHIO R AT NEW CUMBERLAND LOCK & DAM (UPPER), OH",40.528399,-80.626740,23820.0,652.06,None,NaN,NaN,NaN,2010-09-30,2018-01-25,2659.0,https://waterdata.usgs.gov/nwis/dv?cb_00065=on...,gage
03110690,Ohio,"OHIO R AT NEW CUMBERLAND LOCK & DAM (LOWER), OH",40.528121,-80.625629,23820.0,631.56,None,NaN,NaN,NaN,2010-09-30,2018-01-25,2630.0,https://waterdata.usgs.gov/nwis/dv?cb_00065=on...,gage
03111515,Ohio,"OHIO R AT PIKE ISLAND DAM NR WHEELING (UPPER), WV",40.152849,-80.699802,24600.0,631.52,None,NaN,NaN,NaN,2010-09-30,2018-01-25,2662.0,https://waterdata.usgs.gov/nwis/dv?cb_00065=on...,gage


In [369]:
df.type.value_counts()

discharge    103
gage          25
Name: type, dtype: int64

In [386]:
dates = pd.date_range(start_date, end_date)
df_data = pd.DataFrame(index=dates)
df_data.head()

""
2011-10-01
2011-10-02
2011-10-03
2011-10-04
2011-10-05


In [387]:
for site_no in df.index:
    url = df.loc[site_no,'url']
    print('Retrieving data from %s'%url)
    # get data from url using BS
    page = requests.get(url).text
    soup = BeautifulSoup(page,"lxml")
    lines = soup.find(text=re.compile('USGS')).splitlines()
    data = [line.split('\t') for line in lines if line.split('\t')[0]=='USGS']
    columns = [line.split('\t') for line in lines if line.split('\t')[0]=='agency_cd']
    df_station = pd.DataFrame(data,columns=columns[0])
    # clean up raw data
    for col in df_station.columns:
        if col[-5:]=='00003': # select "daily mean" entries
            break
    df_station = df_station[['datetime',col]]
    df_station[col] = pd.to_numeric(df_station[col],errors='coerce')
    df_station.rename(columns={col:site_no,'datetime':'date'},inplace=True)
    df_station = df_station.set_index('date')
    df_station.index = pd.to_datetime(df_station.index)
    # merge into main dataframe
    df_data = pd.concat([df_data,df_station], axis=1)
df_data.to_pickle('USGS_raw_data.pickle')


Retrieving data from https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=02111500&referred_module=sw&period=&begin_date=2011-10-01&end_date=2017-09-30
Retrieving data from https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=03086000&referred_module=sw&period=&begin_date=2011-10-01&end_date=2017-09-30
Retrieving data from https://waterdata.usgs.gov/nwis/dv?cb_00065=on&format=rdb&site_no=03110685&referred_module=sw&period=&begin_date=2011-10-01&end_date=2017-09-30
Retrieving data from https://waterdata.usgs.gov/nwis/dv?cb_00065=on&format=rdb&site_no=03110690&referred_module=sw&period=&begin_date=2011-10-01&end_date=2017-09-30
Retrieving data from https://waterdata.usgs.gov/nwis/dv?cb_00065=on&format=rdb&site_no=03111515&referred_module=sw&period=&begin_date=2011-10-01&end_date=2017-09-30
Retrieving data from https://waterdata.usgs.gov/nwis/dv?cb_00065=on&format=rdb&site_no=03111520&referred_module=sw&period=&begin_date=2011-10-01&end_date=2017-09-30
Retrieving

Retrieving data from https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=05378500&referred_module=sw&period=&begin_date=2011-10-01&end_date=2017-09-30
Retrieving data from https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=05420500&referred_module=sw&period=&begin_date=2011-10-01&end_date=2017-09-30
Retrieving data from https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=05474500&referred_module=sw&period=&begin_date=2011-10-01&end_date=2017-09-30
Retrieving data from https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=05543500&referred_module=sw&period=&begin_date=2011-10-01&end_date=2017-09-30
Retrieving data from https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=05558300&referred_module=sw&period=&begin_date=2011-10-01&end_date=2017-09-30
Retrieving data from https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=05568500&referred_module=sw&period=&begin_date=2011-10-01&end_date=2017-09-30
Retrieving

Retrieving data from https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=07099973&referred_module=sw&period=&begin_date=2011-10-01&end_date=2017-09-30
Retrieving data from https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=07124000&referred_module=sw&period=&begin_date=2011-10-01&end_date=2017-09-30
Retrieving data from https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=07133000&referred_module=sw&period=&begin_date=2011-10-01&end_date=2017-09-30
Retrieving data from https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=07138000&referred_module=sw&period=&begin_date=2011-10-01&end_date=2017-09-30
Retrieving data from https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=07138020&referred_module=sw&period=&begin_date=2011-10-01&end_date=2017-09-30
Retrieving data from https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=07138070&referred_module=sw&period=&begin_date=2011-10-01&end_date=2017-09-30
Retrieving

In [422]:
df_data.describe()

,02111500,03086000,03110685,03110690,03111515,03111520,03112500,03114275,03114280,03150700,...,07263650,07289000,07290880,07331600,07335500,07337000,07344370,07374000,07374525,08266820
count,2190.000000,2192.000000,2176.000000,2150.000000,2183.000000,2168.000000,2149.000000,2174.000000,2173.000000,2183.000000,...,2186.000000,2.191000e+03,969.000000,2192.000000,2192.000000,2175.000000,2192.000000,2.173000e+03,2.145000e+03,2190.000000
mean,151.140228,34991.245438,12.570827,15.246465,12.745318,15.517721,17.648641,12.877898,14.179991,17.661947,...,32.011747,6.657170e+05,35.721889,4781.018431,8989.775091,13088.583908,23500.898723,5.349627e+05,5.259905e+05,67.024338
std,156.542523,26976.851624,0.387422,2.678622,0.231245,3.100420,2.238204,0.176629,2.324297,2.608087,...,1.976571,3.301025e+05,11.093852,10066.992521,19981.561141,25261.199708,31761.111611,2.566121e+05,2.422678e+05,50.502049
min,38.400000,4040.000000,11.850000,12.410000,12.080000,12.220000,15.420000,12.140000,11.710000,15.030000,...,30.940000,1.910000e+05,13.790000,11.400000,177.000000,443.000000,1150.000000,1.410000e+05,7.170000e+04,22.700000
25%,77.825000,14300.000000,12.240000,13.320000,12.590000,13.230000,16.210000,12.760000,12.670000,15.980000,...,31.290000,4.010000e+05,27.090000,160.000000,778.750000,1825.000000,3845.000000,3.200000e+05,3.280000e+05,39.700000
50%,112.000000,27000.000000,12.430000,14.180000,12.750000,14.350000,16.730000,12.900000,13.230000,16.630000,...,31.390000,6.070000e+05,36.050000,783.000000,2100.000000,4270.000000,10200.000000,5.050000e+05,4.910000e+05,48.600000
75%,167.000000,48025.000000,12.920000,16.307500,12.900000,16.762500,18.200000,13.010000,14.830000,18.365000,...,31.540000,8.810000e+05,44.700000,4022.500000,6890.000000,11700.000000,28200.000000,7.070000e+05,7.070000e+05,69.675000
max,2300.000000,152000.000000,13.620000,28.790000,13.400000,31.850000,30.950000,13.320000,28.360000,34.010000,...,46.080000,1.900000e+06,56.690000,70200.000000,223000.000000,254000.000000,206000.000000,1.330000e+06,1.330000e+06,319.000000


In [424]:
# find stations with formatting problems (table from the url doesn't follow standard formatting)
set1 = set(df_data.columns)
set2 = set(df_data.dropna(axis=1,how='all').columns)
set1.difference(set2)

{'07020850'}

In [425]:
subset = dict.fromkeys(watersheds,[])
for ws in watersheds:
    subset[ws] = list(df[df.watershed==ws].index)

In [426]:
# add station with special coding
site_no = '07020850'
url = df.loc[site_no,'url']
print('Retrieving data from %s'%url)
# get data from url using BS
page = requests.get(url).text
soup = BeautifulSoup(page,"lxml")
lines = soup.find(text=re.compile('USGS')).splitlines()
data = [line.split('\t') for line in lines if line.split('\t')[0]=='USGS']
columns = [line.split('\t') for line in lines if line.split('\t')[0]=='agency_cd']
df_station = pd.DataFrame(data,columns=columns[0])
# clean up raw data
for col in df_station.columns:
    if col[-5:]=='30800': # special coding for gage height
        break
df_station = df_station[['datetime',col]]
df_station[col] = pd.to_numeric(df_station[col],errors='coerce')
df_station.rename(columns={col:site_no,'datetime':'date'},inplace=True)
df_station = df_station.set_index('date')
df_station.index = pd.to_datetime(df_station.index)


Retrieving data from https://waterdata.usgs.gov/nwis/dv?cb_00065=on&format=rdb&site_no=07020850&referred_module=sw&period=&begin_date=2011-10-01&end_date=2017-09-30


In [427]:
df_station.head()

,07020850
date,
2011-10-01,19.54
2011-10-02,19.45
2011-10-03,19.07
2011-10-04,18.79
2011-10-05,18.57


In [428]:
dfnew = df_data.dropna(axis=1,how='all')

In [429]:
dfnew = pd.concat([dfnew,df_station],axis=1)
dfnew[site_no]

date
2011-10-01    19.54
2011-10-02    19.45
2011-10-03    19.07
2011-10-04    18.79
2011-10-05    18.57
2011-10-06    18.14
2011-10-07    17.36
2011-10-08    16.74
2011-10-09    15.90
2011-10-10    15.76
2011-10-11    15.47
2011-10-12    14.73
2011-10-13    14.01
2011-10-14    13.58
2011-10-15    13.27
2011-10-16    13.50
2011-10-17    13.45
2011-10-18    13.58
2011-10-19    13.80
2011-10-20    14.04
2011-10-21    14.11
2011-10-22    13.93
2011-10-23    13.52
2011-10-24    13.29
2011-10-25    12.90
2011-10-26    12.89
2011-10-27    12.90
2011-10-28    12.99
2011-10-29    13.24
2011-10-30    13.11
              ...  
2017-09-01    19.06
2017-09-02    18.84
2017-09-03    18.37
2017-09-04    18.03
2017-09-05    17.43
2017-09-06    16.90
2017-09-07    16.54
2017-09-08    16.39
2017-09-09    16.25
2017-09-10    15.89
2017-09-11    15.42
2017-09-12    14.83
2017-09-13    14.42
2017-09-14    13.76
2017-09-15    13.10
2017-09-16    12.87
2017-09-17    12.52
2017-09-18    12.26
2017-09-19    1

In [430]:
dfnew.columns

Index(['02111500', '03086000', '03110685', '03110690', '03111515', '03111520',
       '03112500', '03114275', '03114280', '03150700',
       ...
       '07289000', '07290880', '07331600', '07335500', '07337000', '07344370',
       '07374000', '07374525', '08266820', '07020850'],
      dtype='object', length=128)

In [431]:
dfnew.to_pickle('USGS_raw_data.pickle')

In [434]:
# get turbidity data
url = 'https://waterdata.usgs.gov/nwis/dv?cb_63680=on&format=rdb&site_no=07374000&referred_module=sw&period=&begin_date=2011-10-01&end_date=2017-09-30'
print('Retrieving data from %s'%url)
# get data from url using BS
page = requests.get(url).text
soup = BeautifulSoup(page,"lxml")
lines = soup.find(text=re.compile('USGS')).splitlines()
data = [line.split('\t') for line in lines if line.split('\t')[0]=='USGS']
columns = [line.split('\t') for line in lines if line.split('\t')[0]=='agency_cd']
df_station = pd.DataFrame(data,columns=columns[0])
# clean up raw data
for col in df_station.columns:
    if col[-5:]=='00003': # special coding for gage height
        break
df_station = df_station[['datetime',col]]
df_station[col] = pd.to_numeric(df_station[col],errors='coerce')
df_station.rename(columns={col:'turbidity','datetime':'date'},inplace=True)
df_station = df_station.set_index('date')
df_station.index = pd.to_datetime(df_station.index)

Retrieving data from https://waterdata.usgs.gov/nwis/dv?cb_63680=on&format=rdb&site_no=07374000&referred_module=sw&period=&begin_date=2011-10-01&end_date=2017-09-30


In [435]:
df_station.head()

,turbidity
date,
2011-10-01,21.4
2011-10-02,22.4
2011-10-03,22.6
2011-10-04,22.7
2011-10-05,22.5


In [436]:
dfnew = pd.read_pickle('USGS_raw_data.pickle')
dfnew = pd.concat([dfnew,df_station],axis=1)

In [437]:
dfnew.head()

,02111500,03086000,03110685,03110690,03111515,03111520,03112500,03114275,03114280,03150700,...,07290880,07331600,07335500,07337000,07344370,07374000,07374525,08266820,07020850,turbidity
date,,,,,,,,,,,,,,,,,,,,,
2011-10-01,53.4,17700,12.70,13.48,12.71,13.73,16.33,12.99,12.61,NaN,...,NaN,103.0,482,635.0,1430,274000.0,290000.0,37.3,19.54,21.4
2011-10-02,51.9,24900,12.38,13.79,12.63,14.04,16.40,12.87,13.27,NaN,...,NaN,103.0,316,619.0,1390,278000.0,289000.0,41.0,19.45,22.4
2011-10-03,51.4,42600,12.17,15.58,12.51,16.18,17.76,13.00,14.42,NaN,...,NaN,102.0,278,635.0,1370,282000.0,289000.0,37.6,19.07,22.6
2011-10-04,51.4,56700,12.09,16.94,12.56,17.23,18.30,12.68,15.11,NaN,...,NaN,101.0,245,760.0,1350,286000.0,293000.0,34.2,18.79,22.7
2011-10-05,51.0,54700,12.21,16.70,12.67,17.13,18.27,12.83,14.83,18.28,...,NaN,103.0,223,762.0,1330,292000.0,281000.0,39.3,18.57,22.5


In [438]:
dfnew.to_pickle('USGS_raw_data.pickle')